## A Widget for Inspecting Highlighting
Manually testing syntax highlighting is one approach for catching regressions. This widget will look at a token file created by `Get Cell Source Tokens`, and compare it with a known-good fixture.

If it finds any difference, it will show a link to the sample in question which will pop open a new editor in JupyterLab. If you manually verify the output, you can `Trust` the sample and copy the token file into the expected location.

### So What?
While this may not be _immediately_ useful to you unless you are writing a syntax highlighter, the pattern is useful for all kinds of automation/testing workflows:

- write a test that generates some structured output
- compare the output to a known good example (if available)
- provide a way to incrementally (and interactively!) for a person to improve the fixture data
- repeat the test until you're done

In [1]:
import shutil, re, traitlets as T, IPython.display as D, ipywidgets as W
from pathlib import Path
from difflib import context_diff
from robot.libraries.BuiltIn import BuiltIn

In [2]:
class SyntaxChecker(W.Widget):
    sample_root = T.Instance(Path)
    sample_glob = T.Unicode()
    trusted_root = T.Instance(Path)
    observed_root = T.Instance(Path)
    blacklist = T.Tuple()

    samples = T.Dict()
    trusted = T.Dict()
    observed = T.Dict()
    disagree = T.Dict()
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self._ndlink(["sample_root", "sample_glob"], "samples", self._update_samples)
        self._ndlink(["observed_root"], "observed", self._update_observed)
        self._ndlink(["trusted_root"], "trusted", self._update_trusted)
        self._ndlink(["trusted", "observed"], "disagree", self._update_disagree)
    
    def _ndlink(self, src, dest, handler):
        [T.dlink((self, s), (self, dest), handler) for s in src]
    
    def _rel_stem(self, path, root):
        return str(path.relative_to(root))[:-len(path.suffix)]
    
    def _stem_dict(self, root, pattern):
        if root and pattern:
            return {
                self._rel_stem(p, root): p
                for p in root.glob(pattern)
                if all([b not in str(p) for b in self.blacklist])
            }
        else:
            return {}
        
    
    def _update_samples(self, _):
        return self._stem_dict(self.sample_root, self.sample_glob)
    
    def _update_observed(self, _):
        return self._stem_dict(self.observed_root, "**/*.tokens")
    
    def _update_trusted(self, _):
        return self._stem_dict(self.trusted_root, "**/*.tokens")
    
    def _update_disagree(self, _):
        disagree = {}
        for k, v in self.observed.items():
            observed = v.read_text()
            if k in self.trusted:
                trusted = self.trusted[k].read_text()
            else:
                trusted = ""
            if observed != trusted:
                disagree[k] = "\n".join(
                    context_diff(observed.split("\n"), 
                                 trusted.split("\n")))
        return disagree
    
    def _disagree_row(self, k, v):
        link = W.Output()
        link.layout.flex = "1"
        btn = W.Button(description="Trust?", button_style="primary")
        btn.layout.flex = "0"
        btn.layout.min_width = "10em"
        n = "\n"
        changes = W.HTML(f"""<details>
                          <summary>{len(v.split(n))} changes</summary>
                          <pre>{v}</pre></details>""")
        changes.layout.flex = "1"
        
        @link.capture()
        def _show():
            D.display(D.Markdown(f"[{k}]({self.samples[k]})"))
        
        def _click(btn):
            if btn.description == "Trust?":
                btn.description = "Confirm?"
                btn.button_style = "warning"
            elif btn.description == "Confirm?":
                observed = self.observed[k]
                trusted = (
                    self.trusted_root 
                    / observed.relative_to(self.observed_root)
                )
                trusted.parent.mkdir(parents=True, exist_ok=True)
                shutil.copy(observed, trusted)
                btn.description = "Trusted!"
                btn.button_style = "success"
                
        btn.on_click(_click)
        
        _show()
        box = W.HBox([link, changes, btn])
        box.layout.display = "flex"
        return box
    
    def show_disagree(self):
        return W.VBox([W.HTML("<h2>Untrusted Samples</h2>")] + ([
            self._disagree_row(k, v) for k, v in self.disagree.items()
        ] or [W.HTML("<h3>Nothing to report!</h3>")]))

Not sure if this is actually importable yet, but for now gate behind a `__main__` check for immediate testing.

In [3]:
if __name__ == "__main__":
    try: del checker
    except: pass
    checker = SyntaxChecker(
        sample_root=Path("..") / "fixtures" / "highlighting" / "samples",
        observed_root=Path("../..") / "_testoutput" / "robot" / "linux" / "headlessfirefox" / "36" / "tokens",
        trusted_root=Path("..") / "fixtures" / "highlighting" / "tokens",
        sample_glob="**/*.robot",
        blacklist=["ipynb_checkpoints"]
    )
    try: del disagree
    except: pass
    disagree = checker.show_disagree()
    D.display(disagree)

Let's expose it as a Robot Framework keyword. We'll use the `OperatingSystem` library to normalize the paths. Thanks to `importnb`, the next markdown cell will be the in-browser documentatin of `SyntaxCheckerLibrary.Show Syntax Checker`.

Displays a visual verification tool for validating syntax highlighting specimens.

Loads pairs of like-named token files from `trusted_root` and `observed_root` and shows a clickable link which will open the like-named file from `sample_root` and a button for copying from the `observed` to `trusted`. By default, it looks for a `sample_glob` of `.robot` files, but other syntaxes should work. Additionally, a `blacklist` of file patterns can be provided, which ignores `ipynb_checkpoints` by default.

> _Note: This doesn't work particularly well with `irobotframework` at present, as by the time the widget is displayed, the test directory has already been destroyed: some careful path work could make it work, however. Also, while each syntax excursion is cheap, the setup/teardown cycle is fairly long, making it infeasible for in-the-loop testing_

In [7]:
def show_syntax_checker(sample_root, trusted_root, observed_root, blacklist=["ipynb_checkpoints"], sample_glob="**/*.robot"):
    try:
        osl = BuiltIn().get_library_instance("OperatingSystem")
    except:
        BuiltIn().import_library("OperatingSystem")
        osl = BuiltIn().get_library_instance("OperatingSystem")
    
    checker = SyntaxChecker(
        sample_root=Path(osl.normalize_path(sample_root)),
        observed_root=Path(osl.normalize_path(observed_root)), 
        trusted_root=Path(osl.normalize_path(trusted_root)), 
        blacklist=blacklist,
        sample_glob=sample_glob,
    )
    return checker.show_disagree()